In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Carregando os dados
df_breastcancer = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AM/lista-2/breastcancer.csv")

# Dividindo os dados em características (X) e rótulo (y)
X = df_breastcancer.iloc[:, :-1].values  # 30 primeiras colunas são características
y = df_breastcancer.iloc[:, -1].values.reshape(-1, 1)  # Última coluna é a saída

# Normalizando os dados de entrada
X_normalized = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))


In [ ]:
# Calculando média e desvio padrão
def mean_std(values):
    return np.mean(values), np.std(values)

In [ ]:
def fit_naive_bayes(X_train, y_train):
    classes = np.unique(y_train)
    class_priors = {}
    means = {}
    stds = {}
    for c in classes:
        X_c = X_train[y_train.ravel() == c]
        class_priors[c] = len(X_c) / len(X_train)
        means[c] = np.mean(X_c, axis=0)
        stds[c] = np.std(X_c, axis=0)

    return class_priors, means, stds


In [ ]:
def calculate_probability(x, mean, std):
    exponent = np.exp(-((x - mean) ** 2) / (2 * (std ** 2)))
    return np.prod((1 / (np.sqrt(2 * np.pi) * std)) * exponent)

In [ ]:
def predict_naive_bayes(X_test, class_priors, means, stds):
    predictions = []
    for x in X_test:
        class_probabilities = {}
        for c in class_priors:
            class_probabilities[c] = class_priors[c] * calculate_probability(x, means[c], stds[c])
        predicted_class = max(class_probabilities, key=class_probabilities.get)
        predictions.append(predicted_class)

    return np.array(predictions)

In [ ]:
def evaluate_model_naive_bayes(X_train, y_train, X_test, y_test):
    # Treinando o modelo Naive Bayes
    class_priors, means, stds = fit_naive_bayes(X_train, y_train)

    # Fazendo previsões no conjunto de teste
    y_pred = predict_naive_bayes(X_test, class_priors, means, stds)

    # Calculando acurácia global
    accuracy = np.mean(y_pred == y_test)

    # Calculando a acurácia por classe
    class_accuracy_0 = np.mean((y_pred == y_test)[y_test == 0])
    class_accuracy_1 = np.mean((y_pred == y_test)[y_test == 1])

    return accuracy, class_accuracy_0, class_accuracy_1


In [ ]:
# Definindo Hiperparâmetros
learning_rate = 0.1
iterations = 100
n_folds = 10
fold_size = len(X) // n_folds

# Treinando o modelo Naive Bayes
class_priors, means, stds = fit_naive_bayes(X_normalized, y)

# Realizando a validação cruzada
fold_size = len(X_normalized) // n_folds


# Inicializando as listas para armazenar as métricas de avaliação
global_accuracy_list_nb = []
class_accuracy_0_list_nb = []
class_accuracy_1_list_nb = []

for i in range(n_folds):
    start = i * fold_size
    end = (i + 1) * fold_size
    X_test = X_normalized[start:end]
    y_test = y[start:end].ravel()
    X_train = np.concatenate([X_normalized[:start], X_normalized[end:]])
    y_train = np.concatenate([y[:start], y[end:]])

    accuracy_nb, class_accuracy_0_nb, class_accuracy_1_nb = evaluate_model_naive_bayes(X_train, y_train, X_test, y_test)

    global_accuracy_list_nb.append(accuracy_nb)
    class_accuracy_0_list_nb.append(class_accuracy_0_nb)
    class_accuracy_1_list_nb.append(class_accuracy_1_nb)

# Calculando a média e o desvio padrão das métricas de avaliação
mean_global_accuracy_nb, std_global_accuracy_nb = mean_std(global_accuracy_list_nb)
mean_class_accuracy_0_nb, std_class_accuracy_0_nb = mean_std(class_accuracy_0_list_nb)
mean_class_accuracy_1_nb, std_class_accuracy_1_nb = mean_std(class_accuracy_1_list_nb)


# Imprimindo os resultados
print("Naive Bayes:")
print("Média da acurácia global:", mean_global_accuracy_nb)
print("Desvio padrão da acurácia global:", std_global_accuracy_nb)
print("Média da acurácia para classe 0:", mean_class_accuracy_0_nb)
print("Desvio padrão da acurácia para classe 0:", std_class_accuracy_0_nb)
print("Média da acurácia para classe 1:", mean_class_accuracy_1_nb)
print("Desvio padrão da acurácia para classe 1:", std_class_accuracy_1_nb)